# MASINSKO UCENJE PROJEKAT 3

### Predmet projekta je klasifikacija za video igricu Counter Strike: Global Offensive. Dataset je sacinjen od parametara koji predstavljaju podatke o trenutnom stanju svakog od timova, rezultat klasifikacije je pobeda CT tima (Counter-Terrorist) ili T tima (Terrorist).

## 1. Importovanje potrebnih biblioteka

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import ReduceLROnPlateau

## 2. Ucitavanje podataka

In [2]:
dataset = pd.read_csv("csgo_round_snapshots.csv")
dataset.head()

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,...,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade,round_winner
0,175.00,0.0,0.0,de_dust2,False,500.0,500.0,0.0,0.0,4000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
1,156.03,0.0,0.0,de_dust2,False,500.0,500.0,400.0,300.0,600.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
2,96.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
3,76.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
4,174.97,1.0,0.0,de_dust2,False,500.0,500.0,192.0,0.0,18350.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT


#### Kao sto se moze primetiti dataset se sastoji od 97 featura. Znacajniji su preostalo vreme do kraja runde u sekundama (time_left), trenutni skor u rundama (ct_score, t_score), na kojoj mapi se trenutno igra (map), da li je u zadatom trenutku postavljena C4 bomba (bomb_planted), ukupna vrednost zivota za ceo tim (ct_health, t_health, maksimum je 500=5x100), ukupna vrednost pancira za svaki tim, ukupna vrednost novca, nadalje su zadati parametri koji predstavljaju broj oruzja koje svaki tim poseduje (ct_weapon_"ime_oruzja", t_weapon_"ime_oruzja"), zadati su takodje broj bombi koje tim poseduje. I na kraju je dato koji je tim osvojio rundu.

#### Dalje se vrsi ciscenje dataseta od praznih vrednosti

In [3]:
dataset.dropna(inplace=True)
dataset.reset_index(drop=True, inplace=True)

#### Da bi se izbegi stringovi kao vrednost za map, potrebno je napraviti nove binarne kolone koje ukazuju na to koja je mapa u pitanju 

In [4]:
dataset = pd.get_dummies(dataset,prefix=['map'], columns =['map'])
dataset.head()

,time_left,ct_score,t_score,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,...,t_grenade_decoygrenade,round_winner,map_de_cache,map_de_dust2,map_de_inferno,map_de_mirage,map_de_nuke,map_de_overpass,map_de_train,map_de_vertigo
0,175.00,0.0,0.0,False,500.0,500.0,0.0,0.0,4000.0,4000.0,...,0.0,CT,0,1,0,0,0,0,0,0
1,156.03,0.0,0.0,False,500.0,500.0,400.0,300.0,600.0,650.0,...,0.0,CT,0,1,0,0,0,0,0,0
2,96.03,0.0,0.0,False,391.0,400.0,294.0,200.0,750.0,500.0,...,0.0,CT,0,1,0,0,0,0,0,0
3,76.03,0.0,0.0,False,391.0,400.0,294.0,200.0,750.0,500.0,...,0.0,CT,0,1,0,0,0,0,0,0
4,174.97,1.0,0.0,False,500.0,500.0,192.0,0.0,18350.0,10750.0,...,0.0,CT,0,1,0,0,0,0,0,0


#### Potrebno je prevesti parametar bomb_planted u integer

In [5]:
dataset["bomb_planted"] = dataset["bomb_planted"].astype(int)
dataset.head()

,time_left,ct_score,t_score,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,...,t_grenade_decoygrenade,round_winner,map_de_cache,map_de_dust2,map_de_inferno,map_de_mirage,map_de_nuke,map_de_overpass,map_de_train,map_de_vertigo
0,175.00,0.0,0.0,0,500.0,500.0,0.0,0.0,4000.0,4000.0,...,0.0,CT,0,1,0,0,0,0,0,0
1,156.03,0.0,0.0,0,500.0,500.0,400.0,300.0,600.0,650.0,...,0.0,CT,0,1,0,0,0,0,0,0
2,96.03,0.0,0.0,0,391.0,400.0,294.0,200.0,750.0,500.0,...,0.0,CT,0,1,0,0,0,0,0,0
3,76.03,0.0,0.0,0,391.0,400.0,294.0,200.0,750.0,500.0,...,0.0,CT,0,1,0,0,0,0,0,0
4,174.97,1.0,0.0,0,500.0,500.0,192.0,0.0,18350.0,10750.0,...,0.0,CT,0,1,0,0,0,0,0,0


#### Potrebno je uvesti labele za vrednosti round_winner, da ne bi vise bio string

In [6]:
dataset["round_winner"] = [int(dataset["round_winner"][i]=="CT") for i in range(dataset.shape[0])]

In [7]:
dataset["round_winner"]

0         1
1         1
2         1
3         1
4         1
         ..
122405    0
122406    0
122407    0
122408    0
122409    0
Name: round_winner, Length: 122410, dtype: int64

In [8]:
X = dataset.drop('round_winner', axis=1)
y = dataset['round_winner']
print(X.shape)
print(y.shape)

(122410, 103)
(122410,)


## 3. Obucavanje modela bez normalizacije dataseta

#### Model se sastoji 5 slojeva sa po 64, 32, 16, 8 cvorova i zadnji sloj koji predstavlja predikciju

In [9]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape = (103,) ))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history=model.fit(X, y, epochs=10, batch_size=32, validation_split = .2)

Train on 97928 samples, validate on 24482 samples
Epoch 1/10
97856/97928 [============================>.] - ETA: 0s - loss: 1.4331 - accuracy: 0.6823

c:\users\cile\pycharmprojects\colorreducingunsupervised\venv\lib\site-packages\keras\engine\training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


97928/97928 [==============================] - 12s 125us/sample - loss: 1.4324 - accuracy: 0.6824 - val_loss: 0.5297 - val_accuracy: 0.7234
Epoch 2/10
97928/97928 [==============================] - 11s 117us/sample - loss: 0.6021 - accuracy: 0.6856 - val_loss: 0.6785 - val_accuracy: 0.5822
Epoch 3/10
97928/97928 [==============================] - 12s 118us/sample - loss: 0.6404 - accuracy: 0.6356 - val_loss: 0.6817 - val_accuracy: 0.5631
Epoch 4/10
97928/97928 [==============================] - 11s 116us/sample - loss: 0.6127 - accuracy: 0.6622 - val_loss: 0.5502 - val_accuracy: 0.7087
Epoch 5/10
97928/97928 [==============================] - 12s 118us/sample - loss: 0.5641 - accuracy: 0.7069 - val_loss: 0.5681 - val_accuracy: 0.6782
Epoch 6/10
97928/97928 [==============================] - 11s 117us/sample - loss: 0.5227 - accuracy: 0.7280 - val_loss: 0.5089 - val_accuracy: 0.7187
Epoch 7/10
97928/97928 [==============================] - 11s 113us/sample - loss: 0.5075 - accuracy: 0.7

#### Grafik Test tacnosti za prethodni model

In [ ]:
test_acc = history.history['val_accuracy']
epoch_count = range(1, len(test_acc) + 1)
plt.plot(epoch_count, test_acc, 'b-')
plt.legend(['Test Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
model1 = Sequential()
model1.add(Dense(128, activation='relu', input_shape = (103,) ))
model1.add(Dense(64, activation='relu'))
model1.add(Dense(32, activation='relu'))
model1.add(Dense(16, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))

model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history=model1.fit(X, y, epochs=10, batch_size=32, validation_split = .2)

In [ ]:
test_acc = history.history['val_accuracy']
epoch_count = range(1, len(test_acc) + 1)
plt.plot(epoch_count, test_acc, 'b-')
plt.legend(['Test Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

#### Treniranje modela sa promenjenim learning ratom

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.01)
model1_opt = Sequential()
model1_opt.add(Dense(128, activation='relu', input_shape = (103,) ))
model1_opt.add(Dense(64, activation='relu'))
model1_opt.add(Dense(32, activation='relu'))
model1_opt.add(Dense(16, activation='relu'))
model1_opt.add(Dense(1, activation='sigmoid'))

model1_opt.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history=model1_opt.fit(X, y, epochs=10, batch_size=32, validation_split = .2)

In [ ]:
test_acc = history.history['val_accuracy']
epoch_count = range(1, len(test_acc) + 1)
plt.plot(epoch_count, test_acc, 'b-')
plt.legend(['Test Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.0001)
model1_opt1 = Sequential()
model1_opt1.add(Dense(128, activation='relu', input_shape = (103,) ))
model1_opt1.add(Dense(64, activation='relu'))
model1_opt1.add(Dense(32, activation='relu'))
model1_opt1.add(Dense(16, activation='relu'))
model1_opt1.add(Dense(1, activation='sigmoid'))

model1_opt1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history=model1_opt1.fit(X, y, epochs=10, batch_size=32, validation_split = .2)

In [ ]:
test_acc = history.history['val_accuracy']
epoch_count = range(1, len(test_acc) + 1)
plt.plot(epoch_count, test_acc, 'b-')
plt.legend(['Test Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

## 4. Treniranje nad normalizovanim podacima

In [ ]:
X_normalized=(X-X.min())/(X.max()-X.min()) - 0.5

In [ ]:
X_normalized.head()

In [ ]:
std = StandardScaler()
X_transformed = std.fit_transform(X)
print(X_transformed)

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape = (103,) ))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_normalized, y, epochs=10, batch_size=32, validation_split = .2)

In [ ]:
test_acc = history.history['val_accuracy']
epoch_count = range(1, len(test_acc) + 1)
plt.plot(epoch_count, test_acc, 'b-')
plt.legend(['Test Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.01)
model = Sequential()
model.add(Dense(256, activation='relu', input_shape = (103,) ))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_normalized, y, epochs=10, batch_size=32, validation_split = .2)

In [ ]:
test_acc = history.history['val_accuracy']
epoch_count = range(1, len(test_acc) + 1)
plt.plot(epoch_count, test_acc, 'b-')
plt.legend(['Test Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.00001)
model = Sequential()
model.add(Dense(64, activation='relu', input_shape = (103,) ))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_normalized, y, epochs=10, batch_size=32, validation_split = .2)

In [ ]:
test_acc = history.history['val_accuracy']
epoch_count = range(1, len(test_acc) + 1)
plt.plot(epoch_count, test_acc, 'b-')
plt.legend(['Test Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

#### Treniranje sa dinamickim izborom learning rata

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.0001)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,verbose = 1,
                              patience=5, min_lr=0.00000001)

model = Sequential()
model.add(Dense(1024, activation='relu', input_shape = (103,) ))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_normalized, y, epochs=50, batch_size=32, validation_split = .2, callbacks=[reduce_lr])

In [ ]:
test_acc = history.history['val_accuracy']
epoch_count = range(1, len(test_acc) + 1)
plt.plot(epoch_count, test_acc, 'b-')
plt.legend(['Test Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.000001)
model = Sequential()
model.add(Dense(64, activation='relu', input_shape = (103,) ))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='tanh'))
model.compile(optimizer=opt, loss='hinge', metrics=['accuracy'])

In [ ]:
history = model.fit(X_normalized, y, epochs=10, batch_size=32, validation_split = .2, callbacks=[reduce_lr])

In [ ]:
test_acc = history.history['val_accuracy']
epoch_count = range(1, len(test_acc) + 1)
plt.plot(epoch_count, test_acc, 'b-')
plt.legend(['Test Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()